In [ ]:
#packages

import nltk
#nltk.download('punkt')
#nltk.download('omw-1.4')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('stopwords')
from nltk.corpus import treebank_chunk
import re
from genderize import Genderize
from nltk.corpus import stopwords
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import pickle
import pandas as pd



In [ ]:
#Pick a year, open the pickle and run everything

year = "2010"
file_name = "2010_raw_data.pickle"
file_location = "C:/Users/DanielleDuncan/Documents/Masters-Thesis/2010_raw_data.pickle"

raw_df = pd.read_pickle(file_location)


In [ ]:
def PreProcessing (sentence):
    Male_count = 0
    Female_count = 0
    APIcallfail= 0
    PN_true = None

#regex_cleanup
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub("\d+", " ", sentence)
    sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)
    sentence = re.sub(r"\b's\b", '', sentence)

#tokenize
    sentence =  nltk.TweetTokenizer().tokenize(sentence)
#remove small words
    sentence = [ x for x in sentence if len(x) >= 2 ]

#tag_and_stem
    tagged_sentence = nltk.tag.pos_tag(sentence)
    lemma = nltk.stem.WordNetLemmatizer()
    pn_tags = {'NNP', 'NNPS'}

    new_words = []
    proper_nouns = []
    PN_list = []

    for word, tag in tagged_sentence: 
        if tag not in pn_tags: 
            if tag.startswith("V"):
                lemmas = lemma.lemmatize(word, "v")
            else: 
                lemmas = lemma.lemmatize(word)
            new_words.append((lemmas))
        else:
            proper_nouns.append([word, tag])

    sentence = new_words

#name_gender
    #nltk_results = ne_chunk(tagged_sentence)
    nltk_results = ne_chunk(proper_nouns)

    for nltk_result in nltk_results:
        if type(nltk_result) == Tree:
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            if nltk_result.label() == "PERSON":
                name = name.split(' ')[0]
                PN_list.append(name)
            else: 
                sentence.append(name.strip()) #add a tokenize
    
#Lower
    sentence = [x.lower() for x in sentence]

#contractions
    new_text = []
    for word in sentence:
        contraction = contractions.get(word)
        if contraction is None:
            new_text.append(word)
        else:
            for word in contraction.split():
                new_text.append(word)

    sentence = new_text


#gendered_count
    for w in sentence:
        if w in male_list:
            Male_count += 1
            print(w, "counted male word")
        if w in female_list:
            Female_count += 1


#remove_stopwords
    stops = set(stopwords.words("english"))
    sentence = [x for x in sentence if not x in stops]

#check for PN
    if len(PN_list) > 1:
        PN_true = True

#remove_leakage
    new_sent = [x for x in sentence if x not in male_list]
    new_sent = [x for x in new_sent if x not in female_list]
    sentence = new_sent

    return sentence, Male_count, Female_count, PN_list, PN_true

#if still having memory issues we needd to chunk the dataset 



In [ ]:
#REFERENCE LISTS--TO BE CHANGED
#male_list= #{"abbot", "abbots", "actor", "actors", "author", "authors", "bachelor", "bachelors", "baron", "barons", "boy", "boys", "bridegroom", "bridegrooms", "brother", "brothers", "buck", 
            #"bucks", "conductor", "conductors", "czar", "czars", "dad", "daddy", "daddys", "dads", "duke", "dukes", "emperor", "emperors", "enchanter", "enchanters", "father", "fathers", 
            #"gentleman", "gentlemans", "granddad", "granddads", "grandfather", "grandfathers", "grandpa", "he", "heir", "heirs", "hero", "heros", "hes", "him", "his", "host", "hosts", "hunter", 
            #"hunters", "husband", "husbands", "king", "kings", "landlord", "landlords", "lord", "lords", "man", "mans", "manservant", "manservants", "master", "masters", "men", "milkman", 
            #"milkmans", "mister", "monk", "monks", "mr", "nephew", "nephews", "patron", "patrons", "peacock", "peacocks", "peer", "peers", "poet", "poets", "policeman", "policemans", "policemen", 
            #"policemens", "priest", "priests", "prince", "princes", "prophet", "prophets", "shepherd", "shepherds", "signor", "signors", "sir", "sirs", "son", "songster", "songsters", "sons", 
            #"stag", "stags", "stallion", "stallions", "stepbrother", "stepbrothers", "stepdad", "stepdads", "stepfather", "stepfathers", "steward", "stewards", "sultan", "sultans", "traitor", 
            #"traitors", "uncle", "uncles", "viscount", "viscounts", "waiter", "waiters", "wizard", "wizards"}

#female_list ={#"abbess", "abbesses", "actress", "actresses", "aunt", "aunts", "authoress", "authoresses", "baroness", "baronesses", "benefactress", "benefactresses", "bride", "brides", 
              #"conductress", "conductresses", "countess", "countesses", "czarina", "czarinas", "daughter", "daughters", "duchess", "duchesses", "empress", "empresses", "enchantress", 
              #"enchantresses", "giantess", "giantesses", "girl", "girls", "goddess", "goddesses", "grandma", "grandmas", "grandmother", "grandmothers", "heiress", "heiresses", "hen", 
              #"hens", "her", "heroine", "heroines", "hers", "hostess", "hostesses", "huntress", "huntresses", "ladies", "lady", "landladies", "landlady", "lioness", "lionesses", "madam", 
              #"madams", "maidservant", "maidservants", "milkmaid", "milkmaids", "misses", "missus", "mistress", "mistresses", "mom", "mommy", "mommys", "moms", "mother", "mothers", "mrs", "ms",
              #"mum", "mummy", "mummys", "mums", "murderess", "murderesses", "niece", "nieces", "nun", "nuns", "patroness", "patronesses", "poetess", "poetesses", "policewoman", "policewomen", 
              #"priestess", "priestesses", "princess", "princesses", "prophetess", "prophetesses", "queen", "queens", "she", "shepherdess", "shepherdesses", "shes", "signora", "signoras", 
                #"sister", "sisters", "songstress", "songstresses", "spinster", "spinsters", "stepdaughter", "stepdaughters", "stepmom", "stepmoms", "stepmother", "stepmothers", "stewardess", 
                #"stewardesses", "sultana", "sultanas", "temptress", "temptresses", "tigress", "tigresses", "traitress", "traitresses", "viscountess", "viscountesses", "vixen", "vixens", 
                #"waitress", "waitresses", "wife", "witch", "witches", "wive", "woman", "women"}

male_list = {"man", "boy", "he", "father", "son", "guy", "male", "his", "himself", "grandpa", "grandpas", "grandson", "grandsons", "uncle", "husband", "boy", "brother", "dad", "dude", "fella", "gentleman", "gentlemen", 
             "men", "nephew", "nephews", "sir", "sirs", "lord", "count", "duke", "lad", "mr", "viscount", "daddy", "boys", "guys", "sons", "uncles", "sons", "misters", "mister", "dukes", "daddies", "sons", "lords", 
             "fellas", "stepfather", "stepfathers", "dads", "emperor", "emperors"}
female_list = {"woman", "girl", "she", "mother", "daughter", "gal", "gals", "female", "her", "hers", "herself", "grandma", "grandmas", "granddaughter", "aunt", "wife", "wives", "sister", "sisters", "mum", 
               "mums", "gal", "granny", "lady", "women", "niece", "nieces", "dame", "countess", "duchess", "ladies", "mrs", "ms", "viscountess", "mummy", "girls", "daughters", "aunts", "daughters", "misses", 
               "missus", "grannies", "dames", "mummies", "stepsister", "empress", "empresses", "sisters", "mummys", "stepmother", "stepmothers"}

#Bolukbasi et al repo with words: https://github.com/tolga-b/debiaswe/blob/master/data/gender_specific_full.json

#should all gendered words be counted as a gendered thing???
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
raw_df['encoded_sentences'] = raw_df['sentences'].apply(PreProcessing)

In [ ]:
raw_df_split = pd.DataFrame(raw_df["encoded_sentences"].to_list(), columns=['pre_processed_sent','male_count','female_count','Proper_noun_list', 'pn exists'])
result = pd.concat([raw_df_split, (raw_df.reset_index(drop=True))], axis=1)
result = result.drop("encoded_sentences", axis=1)

In [ ]:
def compare_count(male_col, female_col): 
    """This function compares the count of female to male pronouns. It will output "1" if male count bigger
    than female count, "neutral" if the count is equal, and "female" if there is a higher female count. 
    The function returns strings because we need categorical variables for log reg to run"""
    if female_col > male_col:
        return "1"
    elif male_col > female_col:
        return "0"
    else: 
        return None

In [ ]:
result['col_type'] = result.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)
#lol mistake here
result = result[result["col_type"].notnull()]

In [ ]:
#save the df
new_file_name = year + "_text_wo_names.pickle"

with open(new_file_name, 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)